In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

BATCH_SIZE = 128

model = tf.keras.Sequential()
model.add(layers.Input(shape=(30, 9)))
model.add(layers.LSTM(64, kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [2]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                18944     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 19,074
Trainable params: 19,074
Non-trainable params: 0
_________________________________________________________________
None


In [3]:
def train(model, train_X, train_y):

    dataset = tf.data.Dataset.from_tensor_slices((train_X, train_y))
    dataset = dataset.shuffle(1000).batch(32).repeat()
    steps_per_epoch = train_y.shape[0] // 32

    model.fit(dataset, epochs=50, steps_per_epoch=steps_per_epoch)

In [4]:
def test(model, test_X, test_y):
    predictions = model.predict(test_X).squeeze()
    labels = np.array(test_y).squeeze()
    predictions = np.eye(2)[predictions.argmax(1)]
    acc = accuracy_score(predictions, labels)
    cm = confusion_matrix(predictions.argmax(1), labels.argmax(1))
    print(acc)
    print(cm)


In [5]:
import pickle
with open('fall_down_data', 'rb') as fp:
    X, Y = pickle.load(fp)
print(X.shape)

(338, 30, 9)


In [6]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.1)
train(model, train_X, train_y)

Train for 9 steps
Epoch 1/50
9/9 [==============================] - 3s 343ms/step - loss: 0.7006 - accuracy: 0.5660
Epoch 2/50
9/9 [==============================] - 0s 30ms/step - loss: 0.5021 - accuracy: 0.8088
Epoch 3/50
9/9 [==============================] - 0s 31ms/step - loss: 0.3324 - accuracy: 0.8713
Epoch 4/50
9/9 [==============================] - 0s 31ms/step - loss: 0.2254 - accuracy: 0.9191
Epoch 5/50
9/9 [==============================] - 0s 30ms/step - loss: 0.1736 - accuracy: 0.9338
Epoch 6/50
9/9 [==============================] - 0s 31ms/step - loss: 0.1841 - accuracy: 0.9265
Epoch 7/50
9/9 [==============================] - 0s 31ms/step - loss: 0.0920 - accuracy: 0.9485
Epoch 8/50
9/9 [==============================] - 0s 31ms/step - loss: 0.1080 - accuracy: 0.9449
Epoch 9/50
9/9 [==============================] - 0s 30ms/step - loss: 0.0769 - accuracy: 0.9596
Epoch 10/50
9/9 [==============================] - 0s 32ms/step - loss: 0.0688 - accuracy: 0.9743
Epoch 11/5

In [7]:
import numpy as np
test(model, test_X, test_y)

0.9705882352941176
[[10  1]
 [ 0 23]]


In [8]:
import os
def save_model(model):
    if not os.path.exists('model'):
        os.mkdir('model')
    model.save(os.path.join('model', 'mdoel.h5'))
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.experimental_new_converter = True 
    tflite_model = converter.convert()
    open(os.path.join('model', 'mdoel.tflite'), "wb").write(tflite_model)

In [9]:
save_model(model)